In [3]:
import simpy
import numpy as np

In [28]:
#creating an Arrival class
class Arrival(object):
    def __init__(self, env,arrivalLambda,shuttle):
        self.env = env
        #assigna  shuttle
        self.shuttle = shuttle
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        #set a name by default
        self.arrivalLambda = arrivalLambda
        
    #create a method run that perfomrs the actions when is the enviroment starts running
    def run(self):
        #generate an id for each user
        userID = 0
        while True:
            
            #Generates users with Poisson distribution with lambda
            yield self.env.timeout(np.random.poisson(lam=self.arrivalLambda))
            #instanciate a user
            user = User(env = env, ID = userID,shuttle=self.shuttle) 
            
            print 'User ' + str(user.ID) + ' generated at ' + str(env.now) 
            #increment the user id
            userID +=1

            
            




#Create a user object      
class User(object):
    def __init__(self, env,ID,shuttle):
        self.env = env
        #set a name by default
        self.ID = ID
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        self.shuttle = shuttle
        
    def run(self):
        #moment the user enter the line
        q1_user = env.now

        #waits that the shuttle arrives at point A
        yield self.shuttle.arrivedAtA
        print 'Me user %d ask for a ticket at %d' % (self.ID,env.now)
        q2_user = env.now

        #request for a ticket/resource
        req = shuttleResource.request()
        yield req
        
        yield self.shuttle.arrivedAtB
        print 'Me user %d get down from the shuttle at %d' % (self.ID,env.now)
        q3_user = env.now
        #release de resource
        shuttleResource.release(req)
        
        users.append(self.ID)
        q1.append(q1_user)
        q2.append(q2_user)
        q3.append(q3_user)
        
        
#shuttle class        
class Shuttles(object):
    def __init__(self,env, numOfSeats,tripTime):
        self.env = env
        #Has a capacity
        self.numOfSeats = numOfSeats
        #and a triptime
        self.tripTime = tripTime
        
        #event arrive at point A
        self.arrivedAtA = env.event()
        #event arrive at point B
        self.arrivedAtB = env.event()
        
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

        
    def run(self):
        while True:
            
            print 'Shuttle arrives at A',env.now
            print 'Shuttle leaves from A',env.now
            yield self.env.process(self.going())
            
            print 'Shuttle arrives at B',env.now
            print 'Shuttle leaves from B',env.now
            yield self.env.process(self.returning())
            
            
    #set the events for going and returning
    def going(self):
        #Before the bus leaves A, the event arrived at A happened 
        self.arrivedAtA.succeed()
        #reset the event as pending
        self.arrivedAtA = self.env.event()
        yield self.env.timeout(self.tripTime)
        
    
    def returning(self):
        #Before the bus leaves B, the event arrived at A happened 
        self.arrivedAtB.succeed()
        #reset the event as pending
        self.arrivedAtB = self.env.event()

        yield self.env.timeout(self.tripTime)


  
        
        
#the whole circuit        
class Circuit(object):
    def __init__(self,env,tripTime,arrivalLambda,shuttle):
        self.env = env
        self.tripTime = tripTime
        #set user arrival rate
        self.arrivalLambda = arrivalLambda
        #shuttle 
        self.shuttle = shuttle
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

        
    def run(self):
        #generate one shuttle called shuttle
        #Shuttles(env,numOfSeats=10)
        
        #start to generate users

        bus = Shuttles(env=env, numOfSeats=10,tripTime=15)
        Arrival(env,arrivalLambda = self.arrivalLambda,shuttle=bus)
        yield env.timeout(100)
        
        


In [40]:
#Start the environment
q1 = []
q2 = []
q3 = []
users = []

env = simpy.Environment()
shuttleResource = simpy.Resource(env, capacity=10)

circuito = Circuit(env,tripTime=15,arrivalLambda=2,shuttle='text')
env.run(until=92)

Shuttle arrives at A 0
Shuttle leaves from A 0
User 0 generated at 3
User 1 generated at 6
User 2 generated at 7
User 3 generated at 10
User 4 generated at 13
User 5 generated at 13
User 6 generated at 14
Shuttle arrives at B 15
Shuttle leaves from B 15
User 7 generated at 17
User 8 generated at 19
User 9 generated at 21
User 10 generated at 22
User 11 generated at 22
User 12 generated at 23
User 13 generated at 24
User 14 generated at 26
User 15 generated at 27
User 16 generated at 29
User 17 generated at 30
Shuttle arrives at A 30
Shuttle leaves from A 30
Me user 0 ask for a ticket at 30
Me user 1 ask for a ticket at 30
Me user 2 ask for a ticket at 30
Me user 3 ask for a ticket at 30
Me user 4 ask for a ticket at 30
Me user 5 ask for a ticket at 30
Me user 6 ask for a ticket at 30
Me user 7 ask for a ticket at 30
Me user 8 ask for a ticket at 30
Me user 9 ask for a ticket at 30
Me user 10 ask for a ticket at 30
Me user 11 ask for a ticket at 30
Me user 12 ask for a ticket at 30
Me u

In [46]:
waitingTime = np.array(q2)-np.array(q1)
print 'list of waiting times', waitingTime
print 'average waiting time', waitingTime.mean()

list of waiting times [27 24 23 20 17 17 16 13 11  9  8  8  7  6  4  3  1  0 28 26]
average waiting time 13.4
